In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
from keras.models import load_model
import pickle

In [56]:
# Load dataset test
test_data = pd.read_csv("destinasi_wisata.csv")
test_data

,Unnamed: 0,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Category,city_category
0,0,1,179,3,Candi Ratu Boko,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Yogyakarta,Budaya,Yogyakarta Budaya
1,1,1,344,2,Pantai Marina,"Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, trans...",Semarang,Bahari,Semarang Bahari
2,2,1,5,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Jakarta,Taman Hiburan,Jakarta Taman Hiburan
3,3,1,373,3,Museum Kereta Ambarawa,Museum Kereta Api Ambarawa (bahasa Inggris: In...,Semarang,Budaya,Semarang Budaya
4,4,1,101,4,Kampung Wisata Sosro Menduran,Kampung wisata Sosromenduran merupakan kampung...,Yogyakarta,Budaya,Yogyakarta Budaya
...,...,...,...,...,...,...,...,...,...
9995,9995,300,425,2,Waterpark Kenjeran Surabaya,Waterpark Kenjeran Surabaya merupakan wisata k...,Surabaya,Taman Hiburan,Surabaya Taman Hiburan
9996,9996,300,64,4,Museum Sasmita Loka Ahmad Yani,Museum Sasmita Loka Ahmad Yani adalah salah sa...,Jakarta,Budaya,Jakarta Budaya
9997,9997,300,311,3,The Lodge Maribaya,The Lodge Maribaya adalah salah satu tempat wi...,Bandung,Cagar Alam,Bandung Cagar Alam
9998,9998,300,279,4,Masjid Agung Trans Studio Bandung,Masjid Agung Trans Studio Bandung (TSB) berdir...,Bandung,Tempat Ibadah,Bandung Tempat Ibadah


In [57]:
# Load model
model = load_model("wisata_model.h5")
model

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x0000019511DB6CB0>.

In [58]:
# Menyiapkan dataframe
place_df = test_data[['Place_Id','Place_Name','Category','Place_Ratings']]
place_df.columns = ['id','place_name','category','rating']
place_df[:20]
#df = rating.copy()

,id,place_name,category,rating
0,179,Candi Ratu Boko,Budaya,3
1,344,Pantai Marina,Bahari,2
2,5,Atlantis Water Adventure,Taman Hiburan,5
3,373,Museum Kereta Ambarawa,Budaya,3
4,101,Kampung Wisata Sosro Menduran,Budaya,4
5,312,Taman Hutan Raya Ir. H. Juanda,Cagar Alam,2
6,258,Museum Gedung Sate,Budaya,5
7,20,Museum Taman Prasasti,Budaya,4
8,154,Pantai Ngrawe (Mesra),Bahari,2
9,393,Taman Harmoni Keputih,Cagar Alam,5


In [59]:
# Mengambil sample user
user_id = test_data.User_Id.sample(1).iloc[0]
place_visited_by_user = test_data[test_data.User_Id == user_id]

In [60]:
def dict_encoder(col, data=test_data):

  # Mengubah kolom suatu dataframe menjadi list tanpa nilai yang sama
  unique_val = data[col].unique().tolist()

  # Melakukan encoding value kolom suatu dataframe ke angka
  val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}

  # Melakukan proses encoding angka ke value dari kolom suatu dataframe
  val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
  return val_to_val_encoded, val_encoded_to_val

In [61]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('User_Id')

# Mapping User_Id ke dataframe
test_data['user'] = test_data['User_Id'].map(user_to_user_encoded)

In [62]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Place_Id')

# Mapping Place_Id ke dataframe place
test_data['place'] = test_data['Place_Id'].map(place_to_place_encoded)

In [63]:
# Membuat data lokasi yang belum dikunjungi user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Place_Id.values)]['id'] 
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)
 
place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [64]:
# Membuat data lokasi yang belum dikunjungi user
#place_not_visited = place_df[~place_df['Place_Id'].isin(place_visited_by_user.Place_Id.values)]['Place_Id'] 

### try model

In [65]:
def recommend_places(recommended_place_ids, num_recommendations=7):
    # Prediksi rating menggunakan model
    ratings = model.predict(user_place_array).flatten()
    
    # Urutkan tempat wisata berdasarkan rating tertinggi
    sorted_indices = np.argsort(ratings, axis=0)[-7:][::-1][:num_recommendations]
    
    # Dapatkan informasi tempat wisata yang direkomendasikan
    recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]
    
    return recommended_places

print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)

top_place_user = (
    place_visited_by_user.sort_values(
        by = 'Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)
 
place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)
 
recommended_place = place_df[place_df['id'].isin(top_place_user)]
for row, i in zip(recommended_place.itertuples(), range(1,8)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)
# Preprocessing data test
# Ganti dengan logika preprocessing yang sesuai dengan data test Anda
#test_features = place_df[['id','place_name','category','rating']].values
#test_labels = place_df['id'].values

# Evaluasi model pada data test
#predictions = test_features, test_labels

#predictions = np.array([a[0] for a in predictions])

#predictions

Daftar rekomendasi untuk: User 280

------------------------------------------------------------
Tempat dengan rating wisata paling tinggi dari user
------------------------------------------------------------
Kampung Wisata Kadipaten : Budaya
Taman Pelangi Yogyakarta : Taman Hiburan
Taman Pelangi Yogyakarta : Taman Hiburan
Atlantis Land Surabaya : Taman Hiburan
Taman Pelangi Yogyakarta : Taman Hiburan
Taman Pelangi Yogyakarta : Taman Hiburan
Pantai Sedahan : Bahari
Green Village Gedangsari : Taman Hiburan
Atlantis Land Surabaya : Taman Hiburan
Kampung Wisata Kadipaten : Budaya
Green Village Gedangsari : Taman Hiburan
Taman Pelangi Yogyakarta : Taman Hiburan
Green Village Gedangsari : Taman Hiburan
Taman Pelangi Yogyakarta : Taman Hiburan
Kampung Wisata Kadipaten : Budaya
Kampung Wisata Kadipaten : Budaya
Kampung Wisata Kadipaten : Budaya
Pantai Sedahan : Bahari
Pantai Sedahan : Bahari
Kampung Wisata Kadipaten : Budaya
Kampung Wisata Kadipaten : Budaya
Atlantis Land Surabaya : Taman Hi